In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime as dt

import numpy as np
import pandas as pd

from deep_hedging import (
    Underlyings,
    Ticker,
    ConstantRateCurve,
    EuropeanCall,
)
from deep_hedging.hedger import HedgeSimulator
from deep_hedging.monte_carlo import CorrelatedHestonSimulator
from deep_hedging.monte_carlo.bid_ask import WhiteNoiseBidAskSimulator
from deep_hedging.monte_carlo.volatility import VasicekVolatilitySimulator
from deep_hedging.monte_carlo.rates import VasicekSimulator

RANDOM_SEED = 12

In [3]:
N_PATHS = 10_000

N_STOCKS = 1
TILL_MATURITY = 3.0
TRADING_DAYS = 252
N_DAYS = TRADING_DAYS * TILL_MATURITY

VOL = 0.2

RF_RATE = 0.03
RATES_VOL = 0.0
# RATES_VOL = 0.01

# BID_ASK = 0.05 / 100
BID_ASK = 0.0

In [4]:
np.random.seed(RANDOM_SEED)

start = dt.datetime(2019, 1, 1)
end = start + dt.timedelta(days=N_DAYS)

random_returns = np.concatenate(
    [
        np.expand_dims(np.array([1.0]), 1),
        np.exp(
            (RF_RATE - VOL**2 / 2) / TRADING_DAYS
            + VOL / np.sqrt(TRADING_DAYS) * np.random.randn((end - start).days, N_STOCKS)
        ),
    ],
    axis=0,
)
data = pd.DataFrame(
    random_returns,
    columns=["close"],
)
data["close"] = data["close"].cumprod()
data["index"] = pd.date_range(start=start, end=end, freq="1D")
data.set_index("index", inplace=True)

underlyings = Underlyings(
    tickers=[Ticker(f"Stock {i + 1}", f"{i + 1}") for i in range(N_STOCKS)],
    start=start,
    end=end,
    data=data,
    dividends=np.array([0.0] * N_STOCKS),
)
underlyings.data

,close
index,
2019-01-01,1.000000
2019-01-02,1.006017
2019-01-03,0.997457
2019-01-04,1.000548
2019-01-05,0.979375
...,...
2021-01-22,0.714950
2021-01-23,0.723401
2021-01-24,0.718739


In [5]:
np.random.seed(RANDOM_SEED + RANDOM_SEED)

start = dt.datetime(2019, 1, 1)
end = start + dt.timedelta(days=N_DAYS)

random_returns = np.concatenate(
    [
        np.expand_dims(np.array([RF_RATE]), 1),
        RATES_VOL / np.sqrt(TRADING_DAYS) * np.random.randn((end - start).days, N_STOCKS),
    ],
    axis=0,
)
rates = pd.DataFrame(
    random_returns,
    columns=["close"],
)
rates["close"] = rates["close"].cumsum()
rates["index"] = pd.date_range(start=start, end=end, freq="1D")
rates.set_index("index", inplace=True)
rates

,close
index,
2019-01-01,0.030000
2019-01-02,0.030837
2019-01-03,0.030352
2019-01-04,0.030153
2019-01-05,0.029529
...,...
2021-01-22,0.032307
2021-01-23,0.032221
2021-01-24,0.033141


In [6]:
curve = ConstantRateCurve(constant_rate=RF_RATE)

In [7]:
european_call = EuropeanCall(
    underlyings=underlyings,
    yield_curve=curve,
    strike_level=1.0,
    start_date=start,
    end_date=end,
)

In [8]:
volatility_simulator=VasicekVolatilitySimulator(
    random_seed=RANDOM_SEED,
)
volatility_simulator.fit(underlyings.data)

rates_simulator=VasicekSimulator(
    random_seed=RANDOM_SEED,
)
rates_simulator.fit(rates)

bid_ask_simulator=WhiteNoiseBidAskSimulator(
    random_seed=RANDOM_SEED,
)

In [24]:
rates_simulator.simulate(
    r0=RF_RATE,
    terms=np.arange(1, 100),
    n_paths=100
)

array([[0.03014986, 0.02993396, 0.03001077, ..., 0.0259932 , 0.02553493,
        0.02536495],
       [0.03006271, 0.02963001, 0.02925155, ..., 0.02409529, 0.02390401,
        0.02402042],
       [0.02969614, 0.02919067, 0.02935148, ..., 0.02553612, 0.02553022,
        0.02533328],
       ...,
       [0.02974062, 0.0295788 , 0.0297425 , ..., 0.02883962, 0.02829632,
        0.02857539],
       [0.03029104, 0.03044431, 0.03055268, ..., 0.0292521 , 0.02898863,
        0.02929512],
       [0.03018085, 0.0298678 , 0.02981926, ..., 0.02415457, 0.02398025,
        0.02349161]])

In [9]:
corr_matrix = np.identity(2 * N_STOCKS + 2)
corr_matrix

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [19]:
volatility_simulator.sigma

0.00559340484254123

In [10]:
heston_simulator = CorrelatedHestonSimulator(
    volatility_simulator=volatility_simulator,
    rates_simulator=rates_simulator,
    bid_ask_simulator=bid_ask_simulator,
    payoff_function=european_call.payoff,
    random_seed=RANDOM_SEED,
)

In [27]:
%%time
bids, asks, rates = heston_simulator.get_paths(
    spot=[1.0],
    bid_ask_spread=[BID_ASK],
    rf_rate=RF_RATE,
    vol_start=[VOL],
    bid_ask_spread_var_covar_fn=lambda t: np.array([0] * len(t)),
    time_till_maturity=TILL_MATURITY,
    risk_free_rate_fn=curve.get_instant_fwd_rate,
    dividends_fn=lambda t: 0.0,
    corr_fn=lambda t: np.array([corr_matrix] * len(t)),
    n_paths=N_PATHS,
)
bids.shape, asks.shape, rates.shape

CPU times: user 15.3 s, sys: 235 ms, total: 15.5 s
Wall time: 15.5 s


((10000, 757, 1), (10000, 757, 1), (10000, 756))

In [29]:
hedger = HedgeSimulator(european_call, look_ahead=True)

### Only spot.

In [30]:
simulated_price = hedger.price(
    bids=bids.squeeze(2),
    asks=asks.squeeze(2),
    rates_lend=curve(european_call.days_till_maturity) / TRADING_DAYS,
    rates_borrow=curve(european_call.days_till_maturity) / TRADING_DAYS,
)
simulated_price, european_call.price().item()

(0.4461915030237631, 0.14427105390300576)

### Spot and rates.

In [31]:
simulated_price = hedger.price(
    bids=bids.squeeze(2),
    asks=asks.squeeze(2),
    rates_lend=rates,
    rates_borrow=rates,
)
simulated_price, european_call.price().item()

(13.956583967023558, 0.14427105390300576)

### To model:
* interest rate by HWM
* bid-offer spread, correlated with vol of 
* (Adv.) interest rate, correlated to Sber
* HestonSimulator
* Fit Heston into market via volatility simile using pytorch optimization
* hedging frequency
* LSTMHedger
* bid-offer spread estimate = short option premium from implied vol of option under consideration.